## things to predict:

can you predict a genre given the bands name?

can you predict what decade a rapper/song belongs to?

can you predict what region a rapper/song belongs to? (midwest, south, east, west; maybe even more specific: atlanta, compton, california, NY, detroit)

can you predict artist who are in the rock and roll hall of fame?

topic modeling on best rap albums of all time

## classic rock

https://cache-api.ranker.com/lists/691782/items?limit=50&offset=50&include=votes,wikiText,rankings,openListItemContributors&propertyFetchType=ALL&liCacheKey=null

## alt rock

https://cache-api.ranker.com/lists/336858/items?limit=50&offset=50&include=votes,wikiText,rankings,openListItemContributors&propertyFetchType=ALL&liCacheKey=null&useDefaultNodeLinks=true

# indie folk

https://cache-api.ranker.com/lists/101521/items?limit=50&offset=50&include=votes,wikiText,rankings,openListItemContributors&propertyFetchType=ALL&liCacheKey=null&useDefaultNodeLinks=true

## RnB

https://cache-api.ranker.com/lists/1055563/items?limit=50&offset=50&include=votes,wikiText,rankings,openListItemContributors&propertyFetchType=ALL&liCacheKey=null

## punk rock

https://cache-api.ranker.com/lists/315079/items?limit=50&offset=50&include=votes,wikiText,rankings,openListItemContributors&propertyFetchType=ALL&liCacheKey=null

## screamo

https://cache-api.ranker.com/lists/699150/items?limit=50&offset=50&include=votes,wikiText,rankings,openListItemContributors&propertyFetchType=ALL&liCacheKey=null

## country artists

https://cache-api.ranker.com/lists/588387/items?limit=100&offset=0&include=votes,wikiText,rankings,openListItemContributors&propertyFetchType=ALL&liCacheKey=null

In [13]:
import requests

url = 'https://cache-api.ranker.com/lists/699150/items?limit=100&offset=0&include=votes,wikiText,rankings,openListItemContributors&propertyFetchType=ALL&liCacheKey=null'
r = requests.get(url)
data = r.json()
artists = [i['name'] for i in data['listItems']]

print(artists)

[u'Bring Me The Horizon', u'Asking Alexandria', u'Of Mice & Men', u'A Day To Remember', u'Sleeping With Sirens', u'Falling in Reverse', u'Suicide Silence', u'Motionless In White', u'Black Veil Brides', u'Escape The Fate', u'The Devil Wears Prada', u'Pierce the Veil', u'Slipknot', u'Bullet for My Valentine', u'Memphis May Fire', u'Blessthefall', u'We Came as Romans', u'Panic! at the Disco', u'Miss May I', u'My Chemical Romance', u'Crown the Empire', u'Eyes Set to Kill', u'Five Finger Death Punch', u'Alesana', u'All Time Low', u'August Burns Red', u'Fall Out Boy', u'System of a Down', u'A Skylit Drive', u'The Used', u'Silverstein', u'Nirvana', u'All That Remains', u'Paramore', u'Killswitch Engage', u'Chiodos', u'Atreyu', u'Whitechapel', u'Famous Last Words', u'Attack Attack!', u'Alexisonfire', u'Underoath', u'Get Scared', u'Funeral for a Friend', u'Marilyn Manson', u'Senses Fail', u'Hawthorne Heights', u'Scary Kids Scaring Kids', u'You Me At Six', u'Evanescence', u'Frank Iero', u'Vampire

In [14]:
import sqlalchemy

engine = sqlalchemy.create_engine('')

In [ ]:
import thecypher
import pandas as pd

for artist in artists:
    
    artist_lyrics = thecypher.get_lyrics(artist)
    
    for row in artist_lyrics:
        row['ranker_genre'] = 'screamo'
        
    lyrics_df = pd.DataFrame(artist_lyrics)
    lyrics_df.to_sql('lyrics', con=engine, if_exists='append', index=False)
    print('done ' + artist)

2018-01-30 19:06:24,071 : INFO : GET Artist URL: http://lyrics.wikia.com/wiki/Bring_Me_The_Horizon
2018-01-30 19:06:24,439 : INFO : GET Artist Album: That's the Spirit (2015)
2018-01-30 19:06:29,284 : INFO : GET Artist Album Successful: That's the Spirit (2015)
2018-01-30 19:06:29,285 : INFO : GET Artist Album: Count Your Blessings (2006)
2018-01-30 19:06:34,976 : INFO : GET Artist Album Successful: Count Your Blessings (2006)
2018-01-30 19:06:34,977 : INFO : GET Artist Album: Suicide Season: Cut Up! (2009)
2018-01-30 19:06:40,420 : INFO : GET Artist Album Successful: Suicide Season: Cut Up! (2009)
2018-01-30 19:06:40,420 : INFO : GET Artist Album: Don't Look Down (2014)
2018-01-30 19:06:40,682 : INFO : GET Artist Album Successful: Don't Look Down (2014)
2018-01-30 19:06:40,683 : INFO : GET Artist Album: Sempiternal (2013)
2018-01-30 19:06:45,858 : INFO : GET Artist Album Successful: Sempiternal (2013)
2018-01-30 19:06:45,859 : INFO : GET Artist Album: Misc (0000)
2018-01-30 19:06:45,8

done Bring Me The Horizon


2018-01-30 19:07:09,744 : INFO : GET Artist URL: http://lyrics.wikia.com/wiki/Asking_Alexandria
2018-01-30 19:07:10,079 : INFO : GET Artist Album: Misc (0000)
2018-01-30 19:07:10,080 : INFO : GET Artist Album Successful: Misc (0000)
2018-01-30 19:07:10,081 : INFO : GET Artist Album: Edit
2018-01-30 19:07:18,225 : INFO : GET Artist Album Successful: Edit
2018-01-30 19:07:18,226 : INFO : GET Artist Album: Stand Up and Scream (2009)
2018-01-30 19:07:23,635 : INFO : GET Artist Album Successful: Stand Up and Scream (2009)
2018-01-30 19:07:23,636 : INFO : GET Artist Album: Reckless & Relentless (2011)
2018-01-30 19:07:38,194 : INFO : GET Artist Album Successful: Reckless & Relentless (2011)
2018-01-30 19:07:38,195 : INFO : GET Artist Album: Asking Alexandria (2017)
2018-01-30 19:07:43,586 : INFO : GET Artist Album Successful: Asking Alexandria (2017)
2018-01-30 19:07:43,587 : INFO : GET Artist Album: From Death to Destiny (2013)
2018-01-30 19:07:54,342 : INFO : GET Artist Album Successful: F

done Asking Alexandria


2018-01-30 19:08:01,050 : INFO : GET Artist URL: http://lyrics.wikia.com/wiki/Of_Mice_%26_Men
2018-01-30 19:08:01,373 : INFO : GET Artist Album: Misc (0000)
2018-01-30 19:08:01,374 : INFO : GET Artist Album Successful: Misc (0000)
2018-01-30 19:08:01,374 : INFO : GET Artist Album: Defy (2018)
2018-01-30 19:08:09,063 : INFO : GET Artist Album Successful: Defy (2018)
2018-01-30 19:08:09,064 : INFO : GET Artist Album: Of Mice & Men (2010)
2018-01-30 19:08:21,774 : INFO : GET Artist Album Successful: Of Mice & Men (2010)
2018-01-30 19:08:21,775 : INFO : GET Artist Album: The Flood (2011)
2018-01-30 19:08:30,615 : INFO : GET Artist Album Successful: The Flood (2011)
2018-01-30 19:08:30,616 : INFO : GET Artist Album: Edit
2018-01-30 19:08:30,617 : INFO : GET Artist Album Successful: Edit
2018-01-30 19:08:30,617 : INFO : GET Artist Album: Restoring Force (2014)
2018-01-30 19:08:50,041 : INFO : GET Artist Album Successful: Restoring Force (2014)
2018-01-30 19:08:50,042 : INFO : GET Artist Albu

...

2018-01-30 20:01:18,689 : INFO : GET Artist URL: http://lyrics.wikia.com/wiki/Marilyn_Manson
2018-01-30 20:01:18,999 : INFO : GET Artist Album: Holy Wood (2000)
2018-01-30 20:01:35,091 : INFO : GET Artist Album Successful: Holy Wood (2000)
2018-01-30 20:01:35,091 : INFO : GET Artist Album: Lest We Forget: The Best Of (2004)
2018-01-30 20:01:35,871 : INFO : GET Artist Album Successful: Lest We Forget: The Best Of (2004)
2018-01-30 20:01:35,872 : INFO : GET Artist Album: Misc (0000)
2018-01-30 20:01:35,873 : INFO : GET Artist Album Successful: Misc (0000)
2018-01-30 20:01:35,873 : INFO : GET Artist Album: The Golden Age of Grotesque (2003)
2018-01-30 20:01:45,150 : INFO : GET Artist Album Successful: The Golden Age of Grotesque (2003)
2018-01-30 20:01:45,151 : INFO : GET Artist Album: Smells Like Children (1995)
2018-01-30 20:01:53,456 : INFO : GET Artist Album Successful: Smells Like Children (1995)
2018-01-30 20:01:53,457 : INFO : GET Artist Album: Eat Me, Drink Me (2007)
2018-01-30 20

done Marilyn Manson


2018-01-30 20:03:35,123 : INFO : GET Artist URL: http://lyrics.wikia.com/wiki/Senses_Fail
2018-01-30 20:03:35,430 : INFO : GET Artist Album: Pull The Thorns From Your Heart  (2015)
2018-01-30 20:03:50,916 : INFO : GET Artist Album Successful: Pull The Thorns From Your Heart  (2015)
2018-01-30 20:03:50,917 : INFO : GET Artist Album: From the Depths of Dreams (2002)
2018-01-30 20:03:56,352 : INFO : GET Artist Album Successful: From the Depths of Dreams (2002)
2018-01-30 20:03:56,353 : INFO : GET Artist Album: Life Is Not a Waiting Room (2008)
2018-01-30 20:04:18,588 : INFO : GET Artist Album Successful: Life Is Not a Waiting Room (2008)
2018-01-30 20:04:18,588 : INFO : GET Artist Album: Follow Your Bliss (2012)
2018-01-30 20:04:33,484 : INFO : GET Artist Album Successful: Follow Your Bliss (2012)
2018-01-30 20:04:33,485 : INFO : GET Artist Album: Misc (0000)
2018-01-30 20:04:33,486 : INFO : GET Artist Album Successful: Misc (0000)
2018-01-30 20:04:33,486 : INFO : GET Artist Album: Let It

done Senses Fail


2018-01-30 20:05:40,249 : INFO : GET Artist URL: http://lyrics.wikia.com/wiki/Hawthorne_Heights
2018-01-30 20:05:42,268 : INFO : GET Artist Album: Misc (0000)
2018-01-30 20:05:42,269 : INFO : GET Artist Album Successful: Misc (0000)
2018-01-30 20:05:42,270 : INFO : GET Artist Album: Hurt (2015)
